# **Uploading the Dataset**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sales_dataset.csv to sales_dataset (1).csv


# **Importing the Libraries**

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import shutil
from google.colab import files
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from xgboost.callback import EarlyStopping
import joblib

In [ ]:
!pip install -U kaleido #For exporting plots

# **Reading the Dataset and Standard Summary Statistics**

In [ ]:
#Storing the dataser into a DataFrame
df = pd.read_csv('sales_dataset.csv')

In [ ]:
#Summary Statistics
df.describe()

,Order ID,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
count,9.945990e+05,9.945990e+05,994599.000000,994599.000000,9.945990e+05,9.945990e+05,9.945990e+05
mean,5.493688e+08,4.477692e+05,92.474857,46.323549,3.609370e+07,1.773078e+07,1.836292e+07
std,2.599414e+08,1.508199e+05,80.551447,50.328235,2.969549e+07,1.875209e+07,1.170984e+07
min,1.000012e+08,9.300100e+04,13.500000,3.830000,2.888710e+06,7.636766e+05,1.850556e+06
25%,3.239774e+08,3.439005e+05,27.270000,6.290000,1.400434e+07,3.785686e+06,1.006361e+07
50%,5.487189e+08,4.144380e+05,74.650000,33.120000,2.660794e+07,1.145205e+07,1.464189e+07
75%,7.746139e+08,5.144585e+05,109.970000,66.680000,4.680453e+07,2.462405e+07,2.359062e+07
max,9.999999e+08,1.759186e+06,330.000000,227.310000,2.278949e+08,1.432110e+08,1.091599e+08


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994599 entries, 0 to 994598
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Region          994599 non-null  object 
 1   Country         994599 non-null  object 
 2   Item Type       994599 non-null  object 
 3   Sales Channel   994599 non-null  object 
 4   Order Priority  994599 non-null  object 
 5   Order Date      994599 non-null  object 
 6   Order ID        994599 non-null  int64  
 7   Ship Date       994599 non-null  object 
 8   Units Sold      994599 non-null  int64  
 9   Unit Price      994599 non-null  float64
 10  Unit Cost       994599 non-null  float64
 11  Total Revenue   994599 non-null  float64
 12  Total Cost      994599 non-null  float64
 13  Total Profit    994599 non-null  float64
dtypes: float64(5), int64(2), object(7)
memory usage: 106.2+ MB


In [ ]:
#Checking Duplicates
df.duplicated().sum()

np.int64(0)

In [ ]:
#Converting date columns from strings to datetime data type
df['Ship Date']=pd.to_datetime(df['Ship Date'])
df['Order Date']=pd.to_datetime(df['Order Date'])

In [ ]:
#Dropping Duplicates
df=df.drop_duplicates()

In [ ]:
#Sorting the dataset by the order date for time series analysis and modeling
df=df.sort_values(by='Order Date')

In [ ]:
#Creating shipping delay column for analysis and modeling
df['Shipping Delay'] = (df['Ship Date'] - df['Order Date']).dt.days

In [ ]:
#Checking the columns after the cleaning steps
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 994599 entries, 402719 to 592151
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Region          994599 non-null  object        
 1   Country         994599 non-null  object        
 2   Item Type       994599 non-null  object        
 3   Sales Channel   994599 non-null  object        
 4   Order Priority  994599 non-null  object        
 5   Order Date      994599 non-null  datetime64[ns]
 6   Order ID        994599 non-null  int64         
 7   Ship Date       994599 non-null  datetime64[ns]
 8   Units Sold      994599 non-null  int64         
 9   Unit Price      994599 non-null  float64       
 10  Unit Cost       994599 non-null  float64       
 11  Total Revenue   994599 non-null  float64       
 12  Total Cost      994599 non-null  float64       
 13  Total Profit    994599 non-null  float64       
 14  Shipping Delay  994599 non-null  int

# **Feature Engineering**

**Creating the Modeling df then filtering the targeted items**

In [ ]:
#Getting a copy of the original dataset for modeling
df_modeling = df.copy()

In [ ]:
#Filtering the needed item types
df_modeling = df_modeling[(df_modeling['Item Type']=='Beverages') | (df_modeling['Item Type']=='Meat')].copy()

**Extracting Time Features from the Date**

In [ ]:
# Extract time-based features
df_modeling['Year'] = df_modeling['Order Date'].dt.year
df_modeling['Day'] = df_modeling['Order Date'].dt.day
df_modeling['DayOfWeek'] = df_modeling['Order Date'].dt.dayofweek         # Monday=0, Sunday=6
df_modeling['IsWeekend'] = df_modeling['DayOfWeek'].isin([5, 6]).astype(int)
df_modeling['Quarter'] = df_modeling['Order Date'].dt.quarter
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df_modeling['Season'] = df_modeling['Month'].apply(get_season)

In [ ]:
df_modeling.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165599 entries, 406696 to 553985
Data columns (total 22 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Region          165599 non-null  object        
 1   Country         165599 non-null  object        
 2   Item Type       165599 non-null  object        
 3   Sales Channel   165599 non-null  object        
 4   Order Priority  165599 non-null  object        
 5   Order Date      165599 non-null  datetime64[ns]
 6   Order ID        165599 non-null  int64         
 7   Ship Date       165599 non-null  datetime64[ns]
 8   Units Sold      165599 non-null  int64         
 9   Unit Price      165599 non-null  float64       
 10  Unit Cost       165599 non-null  float64       
 11  Total Revenue   165599 non-null  float64       
 12  Total Cost      165599 non-null  float64       
 13  Total Profit    165599 non-null  float64       
 14  Shipping Delay  165599 non-null  int

**Encoding Categorical Features**

In [ ]:
#Encoding 'Season' as ordinal
season_mapping = {'Spring': 1, 'Summer': 2, 'Fall': 3, 'Winter': 4}
df_modeling['Season_Encoded'] = df_modeling['Season'].map(season_mapping)

#Encoding 'Order Priority' as ordinal
order_priority_mapping = {'L': 1, 'M': 2, 'H': 3, 'C': 4}
df_modeling['Order_Priority_Encoded'] = df_modeling['Order Priority'].map(order_priority_mapping)

#Encoding 'Sales Channel' as binary (Online=1, Offline=0)
df_modeling['Sales_Channel_Encoded'] = df_modeling['Sales Channel'].map({'Online': 1, 'Offline': 0})

#Drop original categorical columns
df_modeling.drop(['Season', 'Order Priority', 'Sales Channel'], axis=1, inplace=True)

In [ ]:
#Target encode 'Country' based on mean of target variable (Units Sold)
country_encoding = df_modeling.groupby('Country')['Units Sold'].mean().to_dict()

#Apply the encoding to the 'Country' column
df_modeling['Country_encoded'] = df_modeling['Country'].map(country_encoding)

In [ ]:
# One-Hot Encoding for Region
df_modeling = pd.get_dummies(df_modeling, columns=['Region'], drop_first=True)

In [ ]:
df_modeling.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165599 entries, 406696 to 553985
Data columns (total 28 columns):
 #   Column                                    Non-Null Count   Dtype         
---  ------                                    --------------   -----         
 0   Country                                   165599 non-null  object        
 1   Item Type                                 165599 non-null  object        
 2   Order Date                                165599 non-null  datetime64[ns]
 3   Order ID                                  165599 non-null  int64         
 4   Ship Date                                 165599 non-null  datetime64[ns]
 5   Units Sold                                165599 non-null  int64         
 6   Unit Price                                165599 non-null  float64       
 7   Unit Cost                                 165599 non-null  float64       
 8   Total Revenue                             165599 non-null  float64       
 9   Total Cost     

In [ ]:
#Checking Columns
df_modeling.columns

Index(['Country', 'Item Type', 'Order Date', 'Order ID', 'Ship Date',
       'Units Sold', 'Unit Price', 'Unit Cost', 'Total Revenue', 'Total Cost',
       'Total Profit', 'Shipping Delay', 'Month', 'Year', 'Day', 'DayOfWeek',
       'IsWeekend', 'Quarter', 'Season_Encoded', 'Order_Priority_Encoded',
       'Sales_Channel_Encoded', 'Country_encoded',
       'Region_Australia and Oceania',
       'Region_Central America and the Caribbean', 'Region_Europe',
       'Region_Middle East and North Africa', 'Region_North America',
       'Region_Sub-Saharan Africa'],
      dtype='object')

**Separate DataFrames for each item**

In [ ]:
#Filter by 'Beverages' and 'Meat' item types
df_beverages = df_modeling[df_modeling['Item Type'] == 'Beverages'].copy()
df_meat = df_modeling[df_modeling['Item Type'] == 'Meat'].copy()

In [ ]:
df_beverages.head()

,Country,Item Type,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,...,Season_Encoded,Order_Priority_Encoded,Sales_Channel_Encoded,Country_encoded,Region_Australia and Oceania,Region_Central America and the Caribbean,Region_Europe,Region_Middle East and North Africa,Region_North America,Region_Sub-Saharan Africa
406696,Saint Lucia,Beverages,2010-01-01,259001243,2010-01-17,519438,14.65,6.02,7609766.70,3127016.76,...,4,1,1,529125.461899,False,True,False,False,False,False
634867,Bhutan,Beverages,2010-01-01,905417478,2010-01-27,505937,15.74,7.21,7963448.38,3647805.77,...,4,4,1,524608.353811,False,False,False,False,False,False
868228,Namibia,Beverages,2010-01-01,408483111,2010-01-17,480706,16.16,5.51,7768208.96,2648690.06,...,4,1,0,540114.640250,False,False,False,False,False,True
597968,Switzerland,Beverages,2010-01-01,313039815,2010-01-24,449462,15.09,6.41,6782381.58,2881051.42,...,4,1,0,518155.835470,False,False,True,False,False,False
365137,Solomon Islands,Beverages,2010-01-01,772308671,2010-01-21,468961,16.23,6.66,7611237.03,3123280.26,...,4,1,0,515181.110609,True,False,False,False,False,False


**Adding Rolling Averages and Lags**

In [ ]:
#Indexing by Order Date for proper Lag and Rolling Average generation
df_beverages = df_beverages.sort_values('Order Date')
df_beverages.set_index('Order Date', inplace=True)

In [ ]:
#Weekly Rolling Average
df_beverages['Rolling_7D'] = df_beverages['Units Sold'].rolling('7D').mean()

In [ ]:
monthly_df_beverages = df_beverages['Units Sold'].resample('ME').sum().to_frame() #Resampling with months to create monthly lags and rolling averages appropriately

#Creating Lags
monthly_df_beverages['Lag_1'] = monthly_df_beverages['Units Sold'].shift(1)
monthly_df_beverages['Lag_3'] = monthly_df_beverages['Units Sold'].shift(3)
monthly_df_beverages['Lag_6'] = monthly_df_beverages['Units Sold'].shift(6)
monthly_df_beverages['Lag_12'] = monthly_df_beverages['Units Sold'].shift(12)

#Creating Rolling Averages
monthly_df_beverages['Rolling_1M'] = monthly_df_beverages['Units Sold'].rolling(1).mean()
monthly_df_beverages['Rolling_3M'] = monthly_df_beverages['Units Sold'].rolling(3).mean()
monthly_df_beverages['Rolling_6M'] = monthly_df_beverages['Units Sold'].rolling(6).mean()
monthly_df_beverages['Rolling_12M'] = monthly_df_beverages['Units Sold'].rolling(12).mean()

In [ ]:
#Merging lags and rolling averages back into the dataset
df_beverages['YearMonth'] = df_beverages.index.to_period('M')
monthly_df_beverages['YearMonth'] = monthly_df_beverages.index.to_period('M')
df_beverages = df_beverages.reset_index().merge(monthly_df_beverages, on='YearMonth', how='left').set_index('Order Date')

In [ ]:
df_beverages.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 82872 entries, 2010-01-01 to 2017-07-29
Data columns (total 38 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Country                                   82872 non-null  object        
 1   Item Type                                 82872 non-null  object        
 2   Order ID                                  82872 non-null  int64         
 3   Ship Date                                 82872 non-null  datetime64[ns]
 4   Units Sold_x                              82872 non-null  int64         
 5   Unit Price                                82872 non-null  float64       
 6   Unit Cost                                 82872 non-null  float64       
 7   Total Revenue                             82872 non-null  float64       
 8   Total Cost                                82872 non-null  float64       
 9   Total Profi

In [ ]:
# Drop rows with NaNs created by lag/rolling features
df_beverages = df_beverages.dropna().reset_index(drop=False)

In [ ]:
#Indexing by Order Date for proper Lag and Rolling Average generation
df_meat = df_meat.sort_values('Order Date')
df_meat.set_index('Order Date', inplace=True)

In [ ]:
#Weekly Rolling Average
df_meat['Rolling_7D'] = df_meat['Units Sold'].rolling('7D').mean()

In [ ]:
monthly_df_meat = df_meat['Units Sold'].resample('ME').sum().to_frame() #Resampling with months to create monthly lags and rolling averages appropriately

#Creating Lags
monthly_df_meat['Lag_1'] = monthly_df_meat['Units Sold'].shift(1)
monthly_df_meat['Lag_3'] = monthly_df_meat['Units Sold'].shift(3)
monthly_df_meat['Lag_6'] = monthly_df_meat['Units Sold'].shift(6)
monthly_df_meat['Lag_12'] = monthly_df_meat['Units Sold'].shift(12)

#Creating Rolling Averages
monthly_df_meat['Rolling_1M'] = monthly_df_meat['Units Sold'].rolling(1).mean()
monthly_df_meat['Rolling_3M'] = monthly_df_meat['Units Sold'].rolling(3).mean()
monthly_df_meat['Rolling_6M'] = monthly_df_meat['Units Sold'].rolling(6).mean()
monthly_df_meat['Rolling_12M'] = monthly_df_meat['Units Sold'].rolling(12).mean()

In [ ]:
#Merging the original meat dataset with the lags and rolling averages
df_meat['YearMonth'] = df_meat.index.to_period('M')
monthly_df_meat['YearMonth'] = monthly_df_meat.index.to_period('M')
df_meat = df_meat.reset_index().merge(monthly_df_meat, on='YearMonth', how='left').set_index('Order Date')

In [ ]:
df_meat.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 82727 entries, 2010-01-01 to 2017-07-29
Data columns (total 38 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Country                                   82727 non-null  object        
 1   Item Type                                 82727 non-null  object        
 2   Order ID                                  82727 non-null  int64         
 3   Ship Date                                 82727 non-null  datetime64[ns]
 4   Units Sold_x                              82727 non-null  int64         
 5   Unit Price                                82727 non-null  float64       
 6   Unit Cost                                 82727 non-null  float64       
 7   Total Revenue                             82727 non-null  float64       
 8   Total Cost                                82727 non-null  float64       
 9   Total Profi

In [ ]:
# Drop rows with NaNs created by lag/rolling features
df_meat = df_meat.dropna().reset_index(drop=False)

In [ ]:
df_beverages.columns

Index(['Order Date', 'Country', 'Item Type', 'Order ID', 'Ship Date',
       'Units Sold_x', 'Unit Price', 'Unit Cost', 'Total Revenue',
       'Total Cost', 'Total Profit', 'Shipping Delay', 'Month', 'Year', 'Day',
       'DayOfWeek', 'IsWeekend', 'Quarter', 'Season_Encoded',
       'Order_Priority_Encoded', 'Sales_Channel_Encoded', 'Country_encoded',
       'Region_Australia and Oceania',
       'Region_Central America and the Caribbean', 'Region_Europe',
       'Region_Middle East and North Africa', 'Region_North America',
       'Region_Sub-Saharan Africa', 'Rolling_7D', 'YearMonth', 'Units Sold_y',
       'Lag_1', 'Lag_3', 'Lag_6', 'Lag_12', 'Rolling_1M', 'Rolling_3M',
       'Rolling_6M', 'Rolling_12M'],
      dtype='object')

In [ ]:
df_meat.columns

Index(['Order Date', 'Country', 'Item Type', 'Order ID', 'Ship Date',
       'Units Sold_x', 'Unit Price', 'Unit Cost', 'Total Revenue',
       'Total Cost', 'Total Profit', 'Shipping Delay', 'Month', 'Year', 'Day',
       'DayOfWeek', 'IsWeekend', 'Quarter', 'Season_Encoded',
       'Order_Priority_Encoded', 'Sales_Channel_Encoded', 'Country_encoded',
       'Region_Australia and Oceania',
       'Region_Central America and the Caribbean', 'Region_Europe',
       'Region_Middle East and North Africa', 'Region_North America',
       'Region_Sub-Saharan Africa', 'Rolling_7D', 'YearMonth', 'Units Sold_y',
       'Lag_1', 'Lag_3', 'Lag_6', 'Lag_12', 'Rolling_1M', 'Rolling_3M',
       'Rolling_6M', 'Rolling_12M'],
      dtype='object')

In [ ]:
# Dropping the duplicate column
df_beverages = df_beverages.drop(columns=['Units Sold_y'])

# Renaming 'Units Sold_x' to 'Units Sold'
df_beverages = df_beverages.rename(columns={'Units Sold_x': 'Units Sold'})

df_meat = df_meat.drop(columns=['Units Sold_y'])
df_meat = df_meat.rename(columns={'Units Sold_x': 'Units Sold'})

In [ ]:
df_beverages = df_beverages.drop(columns=['YearMonth'])
df_meat = df_meat.drop(columns=['YearMonth'])

In [ ]:
df_beverages.columns

Index(['Order Date', 'Country', 'Item Type', 'Order ID', 'Ship Date',
       'Units Sold', 'Unit Price', 'Unit Cost', 'Total Revenue', 'Total Cost',
       'Total Profit', 'Shipping Delay', 'Month', 'Year', 'Day', 'DayOfWeek',
       'IsWeekend', 'Quarter', 'Season_Encoded', 'Order_Priority_Encoded',
       'Sales_Channel_Encoded', 'Country_encoded',
       'Region_Australia and Oceania',
       'Region_Central America and the Caribbean', 'Region_Europe',
       'Region_Middle East and North Africa', 'Region_North America',
       'Region_Sub-Saharan Africa', 'Rolling_7D', 'Lag_1', 'Lag_3', 'Lag_6',
       'Lag_12', 'Rolling_1M', 'Rolling_3M', 'Rolling_6M', 'Rolling_12M'],
      dtype='object')

In [ ]:
#Columns to be dropped before modeling
drop_columns = ['Order Date','Country', 'Item Type', 'Order ID', 'Ship Date', 'Total Revenue', 'Total Cost',
       'Total Profit' ]

In [ ]:
# Dropping unnecessary columns for each item type dataset
df_beverages_cleaned = df_beverages.drop(columns=drop_columns)
df_meat_cleaned = df_meat.drop(columns=drop_columns)

In [ ]:
df_beverages_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71878 entries, 0 to 71877
Data columns (total 29 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Units Sold                                71878 non-null  int64  
 1   Unit Price                                71878 non-null  float64
 2   Unit Cost                                 71878 non-null  float64
 3   Shipping Delay                            71878 non-null  int64  
 4   Month                                     71878 non-null  int32  
 5   Year                                      71878 non-null  int32  
 6   Day                                       71878 non-null  int32  
 7   DayOfWeek                                 71878 non-null  int32  
 8   IsWeekend                                 71878 non-null  int64  
 9   Quarter                                   71878 non-null  int32  
 10  Season_Encoded                    

# **Modeling**

**Train Test Split**

In [ ]:
# Computing 80% split index
split_idx_beverages = int(len(df_beverages_cleaned) * 0.8)
split_idx_meat = int(len(df_meat_cleaned) * 0.8)

# Creating time-based splits
X_train_beverages = df_beverages_cleaned.iloc[:split_idx_beverages].drop(columns='Units Sold')
y_train_beverages = df_beverages.iloc[:split_idx_beverages]['Units Sold']

X_test_beverages = df_beverages_cleaned.iloc[split_idx_beverages:].drop(columns='Units Sold')
y_test_beverages = df_beverages.iloc[split_idx_beverages:]['Units Sold']

X_train_meat = df_meat_cleaned.iloc[:split_idx_meat].drop(columns='Units Sold')
y_train_meat = df_meat.iloc[:split_idx_meat]['Units Sold']

X_test_meat = df_meat_cleaned.iloc[split_idx_meat:].drop(columns='Units Sold')
y_test_meat = df_meat.iloc[split_idx_meat:]['Units Sold']

In [ ]:
X_test_beverages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14376 entries, 57502 to 71877
Data columns (total 28 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Unit Price                                14376 non-null  float64
 1   Unit Cost                                 14376 non-null  float64
 2   Shipping Delay                            14376 non-null  int64  
 3   Month                                     14376 non-null  int32  
 4   Year                                      14376 non-null  int32  
 5   Day                                       14376 non-null  int32  
 6   DayOfWeek                                 14376 non-null  int32  
 7   IsWeekend                                 14376 non-null  int64  
 8   Quarter                                   14376 non-null  int32  
 9   Season_Encoded                            14376 non-null  int64  
 10  Order_Priority_Encoded        

In [ ]:
#Checking if the time ranges are correct after splitting the data
print("Beverages:")
print("Train date range:", df_beverages['Order Date'].iloc[:split_idx_beverages].min(), "to", df_beverages['Order Date'].iloc[:split_idx_beverages].max())
print("Test date range :", df_beverages['Order Date'].iloc[split_idx_beverages:].min(), "to", df_beverages['Order Date'].iloc[split_idx_beverages:].max())

print("\nMeat:")
print("Train date range:", df_meat['Order Date'].iloc[:split_idx_meat].min(), "to", df_meat['Order Date'].iloc[:split_idx_meat].max())
print("Test date range :", df_meat['Order Date'].iloc[split_idx_meat:].min(), "to", df_meat['Order Date'].iloc[split_idx_meat:].max())

Beverages:
Train date range: 2011-01-01 00:00:00 to 2016-04-09 00:00:00
Test date range : 2016-04-09 00:00:00 to 2017-07-29 00:00:00

Meat:
Train date range: 2011-01-01 00:00:00 to 2016-04-03 00:00:00
Test date range : 2016-04-04 00:00:00 to 2017-07-29 00:00:00


**Train Validation Split**

In [ ]:
train_size = int(len(X_train_beverages) * 0.8)  # 80% for training, 20% for validation

# Training set
X_train_ts_beverages, y_train_ts_beverages = X_train_beverages[:train_size], y_train_beverages[:train_size]

# Validation set
X_val_ts_beverages, y_val_ts_beverages = X_train_beverages[train_size:], y_train_beverages[train_size:]

**Tuning the number of decision trees for random forest**

In [ ]:
r2_scores = []

for n in [10, 50, 100, 200, 300, 500, 1000]:
    model = RandomForestRegressor(n_estimators=n, random_state=42)
    model.fit(X_train_ts_beverages, y_train_ts_beverages)
    preds = model.predict(X_val_ts_beverages)
    r2_scores.append(r2_score(y_val_ts_beverages, preds))

In [ ]:
n_estimators = [10, 50, 100, 200, 300, 500, 1000]

# Creating the plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=n_estimators, y=r2_scores, mode='markers+lines',
                         marker=dict(symbol='circle', size=8), line=dict(shape='linear')))

# Setting the labels and title
fig.update_layout(
    title={'text':'Model Performance vs. Number of Trees','x':0.5,'xanchor':'center'},
    xaxis_title='n_estimators',
    yaxis_title='R² Score',
    template='plotly',
    showlegend=False
)

# Showing the plot
fig.show()
figs.append(fig)

**Experimenting with XGBoost and finding the best iteration**

In [ ]:
# Convert data to DMatrix format
dtrain = xgb.DMatrix(X_train_beverages, label=y_train_beverages)
dval = xgb.DMatrix(X_val_ts_beverages, label=y_val_ts_beverages)

# Params
params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.05,
    'max_depth': 6,
    'seed': 42,
    'eval_metric': 'rmse'
}

# Evaluation set
evals = [(dtrain, 'train'), (dval, 'eval')]

# Prepare an empty dict to store results
evals_result = {}

# Train with early stopping and tracking
model = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=evals,
    early_stopping_rounds=50,
    evals_result=evals_result,
    verbose_eval=100
)

# Get best iteration
best_iteration = model.best_iteration
print(f"Best iteration: {best_iteration}")

# Predict
y_pred = model.predict(dval)

# Metrics
r2 = r2_score(y_val_ts_beverages, y_pred)
mse = mean_squared_error(y_val_ts_beverages, y_pred)
rmse = np.sqrt(mean_squared_error(y_val_ts_beverages, y_pred))
print(f"R² Score: {r2}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

[0]	train-rmse:178750.51264	eval-rmse:178545.52384
[100]	train-rmse:101346.21079	eval-rmse:101410.63524
[200]	train-rmse:99094.03978	eval-rmse:99083.22063
[300]	train-rmse:96840.87368	eval-rmse:96699.88561
[400]	train-rmse:94719.70124	eval-rmse:94425.33944
[500]	train-rmse:92739.08336	eval-rmse:92472.37225
[600]	train-rmse:91029.47235	eval-rmse:90773.39139
[700]	train-rmse:89436.29413	eval-rmse:89164.87380
[800]	train-rmse:88083.40623	eval-rmse:87809.77025
[900]	train-rmse:86517.33247	eval-rmse:86094.81856
[999]	train-rmse:84986.78631	eval-rmse:84679.09361
Best iteration: 999
R² Score: 0.7895960211753845
MSE: 7170548736.0
RMSE: 84679.09267345748


**Feature Importance and Learning Curve**

In [ ]:
# Get feature importance from booster
importance_dict = model.get_score(importance_type='gain')

# Convert to DataFrame
importance_df = pd.DataFrame({
    'Feature': list(importance_dict.keys()),
    'Gain': list(importance_dict.values())
})

# Sort by gain
importance_df = importance_df.sort_values(by='Gain', ascending=False)

# Plot
fig = px.bar(importance_df.head(15), x='Gain', y='Feature', orientation='h',
             title='Top Feature Importances (by Gain)', color='Gain', color_continuous_scale='Viridis_r')
fig.update_layout(title={'text':'Top Feature Importances (by Gain)','x':0.5,'xanchor':'center'}, yaxis={'categoryorder':'total ascending'}, height=500)
fig.show()
figs.append(fig)

In [ ]:
# Extract eval results (from evals_result)
results = evals_result

# Create DataFrame for plotting
df_learning = pd.DataFrame({
    'Round': list(range(len(results['train']['rmse']))),
    'Train RMSE': results['train']['rmse'],
    'Validation RMSE': results['eval']['rmse']
})

# Plot learning curve
fig2 = px.line(df_learning, x='Round', y=['Train RMSE', 'Validation RMSE'],
               title='XGBoost Learning Curve (RMSE)', markers=True)
fig2.update_layout(title={'text':'XGBoost Learning Curve (RMSE)','x':0.5,'xanchor':'center'},yaxis_title='RMSE', xaxis_title='Boosting Rounds', height=500)
fig2.show()
figs.append(fig2)

# **Fitting the Model to the Beverages DataFrame**

In [ ]:
# Define the XGBoost Regressor
xgb_model_beverages = xgb.XGBRegressor(
    n_estimators=1000,       # Use the best number of trees from previous tuning
    learning_rate=0.05,      # Optimal learning rate based on previous tests
    random_state=42
)

# Fit the model to the training data
xgb_model_beverages.fit(X_train_beverages, y_train_beverages)

# Make predictions on the test data
y_pred_test_beverages = xgb_model_beverages.predict(X_test_beverages)

In [ ]:
# Calculate R² Score for the test data
r2_test = r2_score(y_test_beverages, y_pred_test_beverages)
print(f'R² Score on Test Data: {r2_test:.3f}')

# Calculate Mean Squared Error for the test data
mse_test = mean_squared_error(y_test_beverages, y_pred_test_beverages)
print(f'Mean Squared Error on Test Data: {mse_test:.3f}')

# Calculate Root Mean Squared Error (RMSE) for the test data
rmse_test = np.sqrt(mse_test)
print(f'Root Mean Squared Error (RMSE) on Test Data: {rmse_test:.3f}')

R² Score on Test Data: 0.642
Mean Squared Error on Test Data: 13868682240.000
Root Mean Squared Error (RMSE) on Test Data: 117765.369


In [ ]:
# Actual vs Predicted Plot
fig_actual_vs_pred = go.Figure()

fig_actual_vs_pred.add_trace(go.Scatter(
    x=y_test_beverages, y=y_pred_test_beverages, mode='markers',
    name='Test Data (Beverages)', marker=dict(color='blue', size=8)
))

fig_actual_vs_pred.update_layout(title={'text':"Actual vs Predicted: Test Data(Beverages)", 'x':0.5, 'xanchor':'center'},
    xaxis_title="Actual Values",
    yaxis_title="Predicted Values",
    showlegend=False
)

fig_actual_vs_pred.show()
figs.append(fig_actual_vs_pred)

In [ ]:
# Residuals Plot
residuals_beverages = y_test_beverages - y_pred_test_beverages

fig_residuals = go.Figure()

fig_residuals.add_trace(go.Scatter(
    x=y_pred_test_beverages, y=residuals_beverages, mode='markers',
    name='Residuals', marker=dict(color='red', size=8)
))

fig_residuals.update_layout(
    title={'text':'Residuals: Test Data (Beverages)','x':0.5,'xanchor':'center'},
    xaxis_title="Predicted Values",
    yaxis_title="Residuals",
    showlegend=False
)

fig_residuals.show()
figs.append(fig_residuals)

In [ ]:
# Get feature importance from booster using 'gain'
importance_dict = xgb_model_beverages.get_booster().get_score(importance_type='gain')

# Convert to DataFrame
importance_df = pd.DataFrame({
    'Feature': list(importance_dict.keys()),
    'Gain': list(importance_dict.values())
})

# Sort by gain in descending order
importance_df = importance_df.sort_values(by='Gain', ascending=False)

# Plot top 15 features
fig = px.bar(importance_df.head(15), x='Gain', y='Feature', orientation='h',
             title='Top Feature Importances by Gain (Beverages)', color='Gain', color_continuous_scale='Viridis_r')

# Update layout
fig.update_layout(title={'text':'Top Feature Importances by Gain (Beverages)','x':0.5,'xanchor':'center'},
                  yaxis={'categoryorder': 'total ascending'}, height=500)
fig.show()
figs.append(fig)

In [ ]:
# Prediction Distribution
fig_pred_dist = go.Figure()

fig_pred_dist.add_trace(go.Histogram(
    x=y_test_beverages, opacity=0.5, name='Actual Values',
    marker=dict(color='blue')
))

fig_pred_dist.add_trace(go.Histogram(
    x=y_pred_test_beverages, opacity=0.5, name='Predicted Values',
    marker=dict(color='red')
))

fig_pred_dist.update_layout(
    title={'text':"Prediction Distribution: Test Data (Beverages)",'x':0.5,'xanchor':'center'},
    xaxis_title="Values",
    yaxis_title="Frequency",
    barmode='overlay',
    showlegend=True
)

fig_pred_dist.show()
figs.append(fig_pred_dist)

**Generating a pickle file for Deployment**

In [ ]:
joblib.dump(xgb_model_beverages,'XGBoost_Beverages_Model.pkl')

['XGBoost_Beverages_Model.pkl']

# **Fitting the Model to the Meat DataFrame**

In [ ]:
# Define the XGBoost Regressor
xgb_model_meat = xgb.XGBRegressor(
    n_estimators=1000,       # Use the best number of trees from previous tuning
    learning_rate=0.05,      # Optimal learning rate based on previous tests
    random_state=42
)

# Fit the model to the training data
xgb_model_meat.fit(X_train_meat, y_train_meat)

# Make predictions on the test data
y_pred_test_meat = xgb_model_meat.predict(X_test_meat)

In [ ]:
# Calculate R² Score for the test data
r2_test_meat = r2_score(y_test_meat, y_pred_test_meat)
print(f'R² Score on Test Data: {r2_test_meat:.3f}')

# Calculate Mean Squared Error for the test data
mse_test_meat = mean_squared_error(y_test_meat, y_pred_test_meat)
print(f'Mean Squared Error on Test Data: {mse_test_meat:.3f}')

# Calculate Root Mean Squared Error (RMSE) for the test data
rmse_test_meat = np.sqrt(mse_test)
print(f'Root Mean Squared Error (RMSE) on Test Data: {rmse_test_meat:.3f}')

R² Score on Test Data: 0.539
Mean Squared Error on Test Data: 3589292544.000
Root Mean Squared Error (RMSE) on Test Data: 117765.369


In [ ]:
# Actual vs Predicted Plot
fig_actual_vs_pred = go.Figure()

fig_actual_vs_pred.add_trace(go.Scatter(
    x=y_test_meat, y=y_pred_test_meat, mode='markers',
    name='Test Data (Beverages)', marker=dict(color='blue', size=8)
))

fig_actual_vs_pred.update_layout(
    title={'text':"Actual vs Predicted: Test Data (Meat)",'x':0.5,'xanchor':'center'},
    xaxis_title="Actual Values",
    yaxis_title="Predicted Values",
    showlegend=False
)

fig_actual_vs_pred.show()
figs.append(fig_actual_vs_pred)

In [ ]:
# Residuals Plot
residuals_meat = y_test_meat - y_pred_test_meat

fig_residuals = go.Figure()

fig_residuals.add_trace(go.Scatter(
    x=y_pred_test_meat, y=residuals_meat, mode='markers',
    name='Residuals', marker=dict(color='red', size=8)
))

fig_residuals.update_layout(
    title={'text':"Residuals: Test Data (Meat)",'x':0.5,'xanchor':'center'},
    xaxis_title="Predicted Values",
    yaxis_title="Residuals",
    showlegend=False
)

fig_residuals.show()
figs.append(fig_residuals)

In [ ]:
# Get feature importance from booster using 'gain'
importance_dict = xgb_model_meat.get_booster().get_score(importance_type='gain')

# Convert to DataFrame
importance_df = pd.DataFrame({
    'Feature': list(importance_dict.keys()),
    'Gain': list(importance_dict.values())
})

# Sort by gain in descending order
importance_df = importance_df.sort_values(by='Gain', ascending=False)

# Plot top 15 features
fig = px.bar(importance_df.head(15), x='Gain', y='Feature', orientation='h',
             title='Top Feature Importances by Gain (Meat)', color='Gain', color_continuous_scale='Viridis_r')

# Update layout
fig.update_layout(title={'text':'Top Feature Importances by Gain (Meat)','x':0.5,'xanchor':'center'}
                  ,yaxis={'categoryorder': 'total ascending'}, height=500)
fig.show()
figs.append(fig)

In [ ]:
# Prediction Distribution
fig_pred_dist = go.Figure()

fig_pred_dist.add_trace(go.Histogram(
    x=y_test_meat, opacity=0.5, name='Actual Values',
    marker=dict(color='blue')
))

fig_pred_dist.add_trace(go.Histogram(
    x=y_pred_test_meat, opacity=0.5, name='Predicted Values',
    marker=dict(color='red')
))

fig_pred_dist.update_layout(
    title={'text':"Prediction Distribution: Test Data (Meat)",'x':0.5,'xanchor':'center'},
    xaxis_title="Values",
    yaxis_title="Frequency",
    barmode='overlay',
    showlegend=True
)

fig_pred_dist.show()
figs.append(fig_pred_dist)

**Generating a pickle file for Deployment purpose**

In [ ]:
joblib.dump(xgb_model_meat,'XGBoost_Meat_Model.pkl')

['XGBoost_Meat_Model.pkl']

**Exporting and Downloading the Plots**

In [ ]:
for f in figs:
    export_plot(f)

 PNG saved: /content/depi_project_plots/distribution_of_total_profit.png
 HTML saved: /content/depi_project_plots/distribution_of_total_profit.html
 PNG saved: /content/depi_project_plots/distribution_of_units_sold.png
 HTML saved: /content/depi_project_plots/distribution_of_units_sold.html
 PNG saved: /content/depi_project_plots/units_sold_per_item_type.png
 HTML saved: /content/depi_project_plots/units_sold_per_item_type.html
 PNG saved: /content/depi_project_plots/monthly_total_revenue.png
 HTML saved: /content/depi_project_plots/monthly_total_revenue.html
 PNG saved: /content/depi_project_plots/monthly_units_sold.png
 HTML saved: /content/depi_project_plots/monthly_units_sold.html
 PNG saved: /content/depi_project_plots/units_sold_by_region.png
 HTML saved: /content/depi_project_plots/units_sold_by_region.html
 PNG saved: /content/depi_project_plots/profit_distribution_by_item_type.png
 HTML saved: /content/depi_project_plots/profit_distribution_by_item_type.html
 PNG saved: /conte

In [ ]:
shutil.make_archive("/content/depi_project_plots", 'zip', "/content/depi_project_plots")
files.download("/content/depi_project_plots.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>